In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts

In [21]:
def initialize_dataset(input_file: str) -> tuple:

    dataframe = pd.read_csv(input_file)
    dataframe  = pd.get_dummies(dataframe ,
                                columns=['gender',
                                         'race/ethnicity',
                                         "parental level of education",
                                         "lunch",
                                         "test preparation course"],
                                prefix=['gender',
                                        'group',
                                        'parent_ed',
                                        'lunch',
                                        'prep_course'], dtype=int)

    # Convert score columns to integers
    dataframe[['math score', 'reading score', 'writing score']] = dataframe[['math score', 'reading score', 'writing score']].astype(int)

    # Create a new column 'passed' based on your condition
    dataframe['passed'] = dataframe.apply(lambda row: 1 if (row['math score'] > 60) & (row['reading score'] > 60) & (row['writing score'] > 60) else -1, axis=1)
    dataframe.drop(['math score', 'reading score', 'writing score'], axis=1, inplace=True)

    # Split the dataset into training and testing sets
    y = dataframe.loc[:, 'passed']
    print("splitting dataset into train and test sets...")
    x_train, x_test, y_train, y_test = tts(dataframe, y, test_size=0.2, random_state=42)

    return (x_train, x_test, y_train, y_test)

In [23]:
input_file = "dataset.csv"
x_train, x_test, y_train, y_test = initialize_dataset(input_file)
x_train.to_csv("x_train.csv", index=False)
x_test.to_csv("x_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

splitting dataset into train and test sets...
